In [ ]:
import numpy as np
import pandas as pd

from is2retreat.config import Params
from is2retreat.pipeline import run_workflow
from is2retreat.bluff import process_cluster_with_reference
from is2retreat.metrics import compute_cluster_statistics


In [ ]:
rows = []
for fam in selected_clusters["gt_family"].unique():
    fam_clusters = selected_clusters.query("gt_family == @fam").copy()

    for cid in fam_clusters["cluster_id"].unique():
        bluff_df, y_ref = process_cluster_with_reference(
            filtered_profiles=filtered_profiles,
            selected_clusters=fam_clusters,
            params=P,
            cluster_id=int(cid),
            gt_family=str(fam),
            which="first",
            bias_df=bias_df,
            debug=False,
        )
        if bluff_df is None or bluff_df.empty:
            continue

        stats = compute_cluster_statistics(
            bluff_df,
            confidence=P.CONFIDENCE,
            min_span_days=P.MIN_SPAN_DAYS,
        )

        angle_val = float(fam_clusters.loc[fam_clusters["cluster_id"] == cid, "angle_deg"].iloc[0]) if "angle_deg" in fam_clusters.columns else np.nan
        center_lat = float(fam_clusters.loc[fam_clusters["cluster_id"] == cid, "center_lat"].iloc[0]) if "center_lat" in fam_clusters.columns else np.nan
        center_lon = float(fam_clusters.loc[fam_clusters["cluster_id"] == cid, "center_lon"].iloc[0]) if "center_lon" in fam_clusters.columns else np.nan

        rows.append({
            "track_id": f"{int(TRACK_ID):04d}",
            "ClusterSize": P.CLUSTER_DISTANCE_M,
            "bias_tolerance": P.BIAS_TOLERANCE,
            "gt_family": fam,
            "cluster_id": int(cid),
            **stats,
            "angle_deg": angle_val,
            "center_lat": center_lat,
            "center_lon": center_lon,
        })

metrics_df = pd.DataFrame(rows)
metrics_df.head()


In [ ]:
metrics_df.to_csv(f"DSAS_metrics_track_{int(TRACK_ID):04d}.csv", index=False)